# Scraping images from mapbox satelitar view  
Mapbox Free tier allows 50,000 requests per month : consider logging how many images have been scraped already to not exceed this limit.  


Pipeline : 
- Define area of interest (bounding box) from coordinates (lat, lon)
- Get the satelitar view of the bounding box from mapbox API
- Crop the images to remove watermark
- Save the images to disk
- Get the bounding boxes of buildings from OSM for the area of interest
- Save the bounding boxes to disk
- Clean overlapping bounding boxes (remove smaller ones or merge them)
- Save the cleaned bounding boxes to disk

The areas of interest have been picked manually :
- Top 20 most populous cities in France
- 100 rural areas coordinates picked randomly across France
- Maybe some random forest to get images without buildings (negative samples)

For the cities : 
- We define a bounding box around the city center with a fixed size (e.g., 10km x 10km)
- We then take random points from this bounding box to scrape images

Then, we can use the pipeline with these points to scrape images and get bounding boxes of buildings.

importing packages

In [2]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

CONFIG

In [ ]:
# Load environment variables from .env file
load_dotenv()

# MAPBOX
MAPBOX_ACCESS_TOKEN = os.getenv("MAPBOX_ACCESS_TOKEN")

# Image parameters
WIDTH = 512
HEIGHT = 512 + 30  # extra space to then delete bottom part with watermark
ZOOM = 18  # 18-19 to see buildings, swimming pools clearly

PATH = Path().cwd().parent
BASE_DIR = PATH / "data"
RAW_DIR = BASE_DIR / "raw_images"
CROPPED_DIR = BASE_DIR / "cropped_images"

RAW_POLYGON_DIR = BASE_DIR / "raw_polygons"
CLEANED_POLYGON_DIR = BASE_DIR / "cleaned_polygons"

CONFIG_DIR = BASE_DIR / ".config_files"

sys.path.append(str(PATH))

First, we get all the coordinates of the areas of interest from the two csv files.

In [4]:
# to import csv files
import pandas as pd

Rural areas coordinates file : "rural_coords.csv"

In [ ]:
RURAL_COORDS_FILE_NAME: Path = CONFIG_DIR / "rural_coords.csv"
rural_coords: pd.DataFrame = pd.read_csv(
    RURAL_COORDS_FILE_NAME,
    sep = ",",
    header=None
)
rural_coords.columns = ["latitude", "longitude"]

In [6]:
# checking the rural coordinates file
print(f"Number of rural areas: {len(rural_coords)}")
rural_coords.head()

Number of rural areas: 100


,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


Cities coordinates file : "city_coords.csv"

In [ ]:
CITY_COORDS_FILE_NAME: Path = CONFIG_DIR / "city_coords.csv"
city_coords: pd.DataFrame = pd.read_csv(
    CITY_COORDS_FILE_NAME,
    sep = ","
)

In [8]:
# checking the city coordinates file
print(f"Number of cities: {len(city_coords)}")
city_coords.head()

Number of cities: 20


,city,lat,lng
0,Paris,48.8567,2.3522
1,Bordeaux,44.8400,-0.5800
2,Marseille,43.2964,5.3700
3,Lyon,45.7600,4.8400
4,Toulouse,43.6045,1.4440


For the cities, we now create a bounding box around the city center with a fixed size (e.g., 10km x 10km) and then take random points from this bounding box to scrape images.

In [9]:
from miscellaneous import create_bbox_from_city_coordinates

city_coords_bbox: dict[str, dict[str, float]] = {}
# {city :  {"west": .., "south": .., "east": .., "north": ..}

for _, row in city_coords.iterrows():
    city_name: str = row["city"]
    latitude: float = row["lat"]
    longitude: float = row["lng"]
    bbox: dict[str, float] = create_bbox_from_city_coordinates(
        latitude,
        longitude,
        width=10,
        height=10
    )
    # width and height are in km
    city_coords_bbox[city_name] = bbox


In [10]:
city_coords_df = pd.DataFrame.from_dict(city_coords_bbox, orient="index")
city_coords_df.head()

,west,south,east,north
Paris,2.284676,48.811347,2.419842,48.902013
Bordeaux,-0.640288,44.792973,-0.519611,44.886994
Marseille,5.306649,43.252774,5.433447,43.339990
Lyon,4.774246,45.716018,4.905859,45.803944
Toulouse,1.383371,43.558618,1.504726,43.650350


Now that we have the bounding boxes for each city, we can get random points from these bounding boxes to scrape images.

In [11]:
from miscellaneous import get_random_points_in_bbox

city_random_points: list[dict[str, float]] = []
for city, bbox in city_coords_bbox.items():
    random_points: list[dict[str, float]] = get_random_points_in_bbox(
        bbox,
        number_of_points=10
    )
    # 10 coordinates/images for each city -> 20 * 10 = 200 images
    city_random_points.extend(random_points)
city_random_points_df = pd.DataFrame(city_random_points)
city_random_points_df.columns = ["latitude", "longitude"]

In [12]:
city_random_points_df.head()

,latitude,longitude
0,48.812470,2.331307
1,48.846397,2.415226
2,48.836093,2.339604
3,48.870927,2.350400
4,48.814249,2.335782


Now that we have all the coordinates (rural + cities), we combine them into a single list of coordinates to use in the pipeline.

In [13]:
# we combine both rural and city coordinates into a single dataframe
all_coords_df = pd.DataFrame(columns=["latitude", "longitude"])
all_coords_df = pd.concat(
    [
        rural_coords,
        city_random_points_df
    ],
    ignore_index=True
)

In [14]:
all_coords_df.head()

,latitude,longitude
0,45.534732,5.164968
1,44.947250,0.813114
2,45.534913,5.180068
3,44.946466,0.804980
4,45.534471,5.185063


We can now follow the pipeline to scrape images and get bounding boxes of buildings.

In [ ]:
# testing on 5 random coordinates from the combined dataframe
coords_to_retrieve = pd.read_csv(CONFIG_DIR / "coords_to_retrieve.csv")
# coords_to_retrieve = all_coords_df.sample(5)
coords_to_retrieve


,latitude,longitude
0,48.836959,2.186961
1,43.325389,5.383074
2,48.842989,2.143196
3,43.745405,7.283145
4,48.611320,7.779747


In [24]:
from miscellaneous import create_bbox_from_coordinates

bboxs: list[dict[str, float]] = []
for _, row in coords_to_retrieve.iterrows():
    # now we get bbox from coordinates
    bbox: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude']
    )
    # other parameters include :
    # - img_height: int = 512 + 30,
    # - img_width: int = 512,
    # - pixel_size: float = 0.4
    # returns : {"west": .., "south": .., "east": .., "north": ..}
    bboxs.append(bbox)

    # print(f"Latitude: {row['latitude']}, Longitude: {row['longitude']}")
print(f"BBox: {bboxs}\n")

BBox: [{'west': 2.185581237015867, 'south': 48.835974680126895, 'east': 2.1883408150949464, 'north': 48.83794330309071}, {'west': 5.381772518276674, 'south': 43.324442084315315, 'east': 5.384375524479792, 'north': 43.3263359007119}, {'west': 2.1418169041314306, 'south': 48.84200418077157, 'east': 2.1445751479869224, 'north': 48.84397380246062}, {'west': 7.281802911148308, 'south': 43.74448094538202, 'east': 7.284487132180541, 'north': 43.74632903863253}, {'west': 7.778271558961928, 'south': 48.610402428253956, 'east': 7.781222492749642, 'north': 48.61223755277122}]



asking the image from mapbox

In [21]:
from miscellaneous import ask_mapbox_for_image

# continuing with the same 5 random coordinates for coords_to_retrieve
for i, row in enumerate(bboxs):    
    # asking mapbox for image
    image = ask_mapbox_for_image(
        image_width_height={"width": WIDTH, "height": HEIGHT},
        mapbox_token=MAPBOX_ACCESS_TOKEN,
        bounding_box=row,
        request_timeout=10,
        output_file=RAW_DIR / f"image_{i+1}.png",  # if output_file is provided, the image will be saved at this location
    )


now cropping the image

In [ ]:
from PIL import Image

# cropping the images to remove the watermark : removing 30 pixels from the bottom
box = (0, 0, WIDTH, HEIGHT - 30)  # left, upper, right, lower
for image_path in RAW_DIR.glob("*.png"):
    # reading the image
    image = Image.open(image_path)
    cropped_image = image.crop(box)
    cropped_image.save(CROPPED_DIR / image_path.name)


Now that we have the cropped image, we can get the polygons on the same coordinates

In [25]:
# we take the same 5 random coordinates and get the corresponding OSM data to check if there are buildings in the area

bboxs_for_polygons: list[dict[str, float]] = []
for _, row in coords_to_retrieve.iterrows():
    # now we get bbox from coordinates
    bbox: dict[str, float] = create_bbox_from_coordinates(
        row['latitude'],
        row['longitude'],
        img_height=512,
        img_width=512,
        pixel_size=0.4
    )
    bboxs_for_polygons.append(bbox)

    # print(f"Latitude: {row['latitude']}, Longitude: {row['longitude']}")
print(f"BBox: {bboxs_for_polygons}\n")


BBox: [{'west': 2.185580393955816, 'south': 48.83602863750833, 'east': 2.188341655328465, 'north': 48.83788934578225}, {'west': 5.381774747940173, 'south': 43.324496034757026, 'east': 5.384373292311113, 'north': 43.3262819502476}, {'west': 2.141816015666921, 'south': 48.84205813754506, 'east': 2.1445760336276094, 'north': 48.843919845759295}, {'west': 7.281806947548115, 'south': 43.74453485706166, 'east': 7.28448309313005, 'north': 43.74627512698972}, {'west': 7.778276481046147, 'south': 48.61045630016747, 'east': 7.78121756746691, 'north': 48.61218368105847}]



getting the polygons for the new bboxs_for_polygons

In [ ]:
# first we import the OpenStreetMap mapping dict we created to have the mapping between OSM tags and our labels
import json

with open(CONFIG_DIR / "osm_to_category_mapping.json", "r") as f:
    mapping_dict = json.load(f)

In [59]:
# printing the 5 first keys of mapping_dict
print(f"Keys in mapping_dict: {list(mapping_dict.keys())[:5]}")
print(f"Values in mapping_dict: {list(mapping_dict.values())[:5]}")

Keys in mapping_dict: ['house', 'semidetached_house', 'terrace', 'cabin', 'bungalow']
Values in mapping_dict: ['Maison', 'Maison', 'Maison', 'Maison', 'Maison']


In [ ]:
import osmnx as ox

for i, bbox in enumerate(bboxs_for_polygons):
    # we get the OSM data for the bbox
    # IMPORTANT: OSMnx expects bbox in order (west, south, east, north)
    bbox_for_ox = (bbox["west"], bbox["south"], bbox["east"], bbox["north"])
    features_from_bbox = ox.features_from_bbox(bbox=bbox_for_ox, tags={"building": True})

    features_from_bbox = features_from_bbox[features_from_bbox["building"].isin(mapping_dict.keys())]
    # we add our category labels to the features
    features_from_bbox["category_of_building"] = features_from_bbox["building"].map(mapping_dict)
    # saving the features as a geojson file
    features_from_bbox.to_file(RAW_POLYGON_DIR / f"features_{i+1}.geojson", driver="GeoJSON")

    print(f"Features for bbox {i+1} saved at {RAW_POLYGON_DIR / f'features_{i+1}.geojson'}")


Features for bbox 1 saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\raw_polygons\features_1.geojson
Features for bbox 2 saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\raw_polygons\features_2.geojson
Features for bbox 3 saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\raw_polygons\features_3.geojson
Features for bbox 4 saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\raw_polygons\features_4.geojson
Features for bbox 5 saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\raw_polygons\features_5.geojson


Now that we have scrapped the raw polygons, we need to "clean" them up.

In [ ]:
import geopandas as gpd
from miscellaneous import clean_overlapping_bboxes

for file in RAW_POLYGON_DIR.glob("*.geojson"):
    gdf = gpd.read_file(file)
    gdf_cleaned = clean_overlapping_bboxes(gdf, threshold=0.3)
    # saving the cleaned features as a geojson file
    gdf_cleaned.to_file(CLEANED_POLYGON_DIR / f"cleaned_{file.name}", driver="GeoJSON")

    # print(f"Features in {file.name}:")
    # print(gdf[["building", "category_of_building"]].head())
    # print(f"Cleaned features for {file.name} saved at {CLEANED_POLYGON_DIR / f'cleaned_{file.name}'}")


c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\.venv\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: Several features with id = 279131445 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


Features in features_1.geojson:
  building category_of_building
0    house               Maison
1      yes        Non_classifie
2    house               Maison
3    house               Maison
4    house               Maison
Cleaned features for features_1.geojson saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\cleaned_polygons\cleaned_features_1.geojson
Features in features_2.geojson:
  building category_of_building
0      yes        Non_classifie
1      yes        Non_classifie
2      yes        Non_classifie
3      yes        Non_classifie
4      yes        Non_classifie
Cleaned features for features_2.geojson saved at c:\Users\olivi\Documents\GitHub\SISE_satelitar_identifier\data\cleaned_polygons\cleaned_features_2.geojson
Features in features_3.geojson:
  building category_of_building
0    house               Maison
1      yes        Non_classifie
2      yes        Non_classifie
3      yes        Non_classifie
4      yes        Non_classifie
Cleaned features

---